In [ ]:
# DATA SUBSETTING AND MERGING

import pandas as pd
import numpy as np

# --- Load raw CSVs ---
games = pd.read_csv("data/games.csv", low_memory=False)
recs  = pd.read_csv("data/recommendations.csv", low_memory=False)
users = pd.read_csv("data/users.csv", low_memory=False)

print("Raw shapes:")
print("games:", games.shape)
print("recommendations:", recs.shape)
print("users:", users.shape)

# --- Subset users who wrote ≥20 reviews ---
user_counts = recs['user_id'].value_counts()
active_users = user_counts[user_counts >= 20].index
recs = recs[recs['user_id'].isin(active_users)]

# --- Subset games that received ≥1500 reviews ---
game_counts = recs['app_id'].value_counts()
popular_games = game_counts[game_counts >= 1500].index
recs = recs[recs['app_id'].isin(popular_games)]

print("After subsetting:")
print("recommendations:", recs.shape)

# --- Merge datasets ---
merged = (
    recs
    .merge(games, on="app_id", how="left")
    .merge(users, on="user_id", how="left")
)

print("Merged shape:", merged.shape)

Raw shapes:
games: (50872, 13)
recommendations: (41154794, 8)
users: (14306064, 3)
After subsetting:
recommendations: (5217474, 8)
Merged shape: (5217474, 22)


In [7]:
# DATA CLEANING

# --- Remove duplicate user–game pairs ---
merged = merged.drop_duplicates(subset=["user_id", "app_id"])

# --- Drop unnecessary columns to reduce size ---
# keep only essential metadata for analysis
cols_to_keep = [
    "user_id", "app_id", "is_recommended", "hours", "helpful", "funny", "date",
    "title", "date_release", "rating", "positive_ratio", "user_reviews",
    "price_final", "discount", "win", "mac", "linux", "steam_deck"
]
merged = merged[[c for c in cols_to_keep if c in merged.columns]]

# --- Convert “is_recommended” to binary ---
merged["is_recommended"] = merged["is_recommended"].map({True: 1, False: 0, "True": 1, "False": 0}).fillna(0).astype(int)

# --- Handle missing numeric or date values ---
if "price_final" in merged.columns:
    median_price = merged["price_final"].median()
    merged["price_final"] = merged["price_final"].fillna(median_price)

if "date_release" in merged.columns:
    merged["date_release"] = pd.to_datetime(merged["date_release"], errors="coerce")
    merged["date_release"] = merged["date_release"].fillna(pd.Timestamp("2000-01-01"))

# --- Optional: fill missing text fields ---
merged["title"] = merged["title"].fillna("Unknown Title")

print("After cleaning:", merged.shape)
print(merged.info(memory_usage='deep'))

After cleaning: (5217467, 18)
<class 'pandas.core.frame.DataFrame'>
Index: 5217467 entries, 0 to 5217473
Data columns (total 18 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         int64         
 1   app_id          int64         
 2   is_recommended  int64         
 3   hours           float64       
 4   helpful         int64         
 5   funny           int64         
 6   date            object        
 7   title           object        
 8   date_release    datetime64[ns]
 9   rating          object        
 10  positive_ratio  int64         
 11  user_reviews    int64         
 12  price_final     float64       
 13  discount        float64       
 14  win             bool          
 15  mac             bool          
 16  linux           bool          
 17  steam_deck      bool          
dtypes: bool(4), datetime64[ns](1), float64(3), int64(7), object(3)
memory usage: 1.6 GB
None


In [8]:
# Save smaller cleaned dataset

merged.to_csv("subset_cleaned.csv", index=False)

print("Cleaned subset saved successfully!")

Cleaned subset saved successfully!


In [9]:
# Number of unique users
n_users = merged["user_id"].nunique()

# Number of unique games
n_games = merged["app_id"].nunique()

# Number of total reviews (edges in the bipartite graph)
n_edges = len(merged)

print(f"Number of unique users: {n_users}")
print(f"Number of unique games: {n_games}")
print(f"Number of user–game pairs (edges): {n_edges}")
print(f"Average reviews per user: {n_edges / n_users:.2f}")
print(f"Average reviews per game: {n_edges / n_games:.2f}")


Number of unique users: 213157
Number of unique games: 1149
Number of user–game pairs (edges): 5217467
Average reviews per user: 24.48
Average reviews per game: 4540.88
